In [49]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress,t,tstd,tmean

In [50]:
folderpath  = '/Users/andreaithompson/Documents/ResearchBMR/MojdehTests/YoungModulus/JunTests/RawData'
excelPath = '/Users/andreaithompson/Documents/ResearchBMR/MojdehTests/YoungModulus/JunTests/Excels/'
graphPath = '/Users/andreaithompson/Documents/ResearchBMR/MojdehTests/YoungModulus/JunTests/Graphs/graph1'

In [51]:
stack_height = np.array([1.92, 2.535, 2.507, 2.722, 3.495, 3.78, 2.784, 3.016, 2.911, 3.182])
area = 11*11 #mm^2 
folderNames = sorted(os.listdir(folderpath))
folderNames.remove('.DS_Store')
#numFiles = 4
#numFiles = len(fileNames)
#print(len(folderNames))

In [86]:
graphRanges = np.array([[0.00,0.01],[0.005,1.0]]) #kN
numRanges = len(graphRanges)
#print(numRanges)
statYM = np.zeros((numRanges,2))

In [87]:
'''determine where the peak of the force/displace curve '''
def peak(i,k):
    #print(folderNames[k])
    #print(fileNames[i])
    #fileNames = sorted(os.listdir(folderpath + '/'+ folderNames[k])) #print(fileNames[i])
    data = pd.read_csv(folderpath + '/'+ folderNames[k] + '/'+ fileNames[i]) #print((folderpath + '/'+ folderNames[k] + '/'+ fileNames[i]))
    raw_data = np.zeros((2,len(data)-1))
    for j in range(len(data)-1):
        raw_data[0][j] = float(data["Displacement"][j+1]) 
        raw_data[1][j] = float(data["Force"][j+1])     
    s = 0.01
    for j in range(len(raw_data[1])):
        if raw_data[1][j] > s:
            s = j
    return s

In [88]:
'''filters unwanted region of the force/displacement curve'''
def curve(i,k,bottom,top):
    #fileNames = sorted(os.listdir(folderpath + '/'+ folderNames[k]))
    data = pd.read_csv(folderpath + '/'+ folderNames[k] + '/'+ fileNames[i])
    #print(len(data))
    raw_data = np.zeros((2,len(data)-1))
    for j in range(len(data)-1):
        raw_data[0][j] = float(data["Displacement"][j+1]) 
        raw_data[1][j] = float(data["Force"][j+1])  
    s = int(peakpoint[i])
    p_data = np.zeros((2,s))
    for j in range(s):
        p_data[0][j] = raw_data[0][j]
        p_data[1][j] = raw_data[1][j]
    o = 0 
    for j in range(len(p_data[1])-1):
        if p_data[1][j] > bottom:
            o = np.append(o,j)
    #e = (o[len(o)-1]-o[1])
    #print(o)
    #print(o[1])
    e = o[1]
    h = 0
    for j in range(len(p_data[1])-1):
        if p_data[1][j] < top:
            h = np.append(h,j)  
    d = h[len(h)-1]
    return [e,d]

In [89]:
'''calculates the stress and the strain'''

def ss(i,k,dd):
    t,b = dd
    t = int(t)
    b = int(b)
    d = b - t
    #fileNames = os.listdir(folderpath + '/'+ folderNames[k])
    #print(folderNames[k])
    #print(fileNames[i])
    data = pd.read_csv(folderpath + '/'+ folderNames[k] + '/'+ fileNames[i])
    raw_data = np.zeros((2,len(data)-1))
    for j in range(len(data)-1):
        raw_data[0][j] = float(data["Displacement"][j+1]) 
        raw_data[1][j] = float(data["Force"][j+1])  
        
    p_ss = np.zeros((2,d))
    for j in range(d):
        p_ss[0][j] = raw_data[0][j + t]/stack_height[k] #strain          
        p_ss[1][j] = raw_data[1][j + t]/area #stress 
    return p_ss

In [90]:
'''function that calculates YM and gives the r value'''

def ym(i,k):
    z = linregress(ss(i,k,bounds[i]))
    return [z[0],z[2]]

In [91]:
'''function that graphs the region which the YM is being calculated for. 
 This function also saves the graph '''

def graphs(k,i,j):
    #fileNames = os.listdir(folderpath + '/'+ folderNames[k])
    a = str(j+1)
    a = "bound" + a
    z = linregress(ss(i,k,bounds[i]))
    plt.figure(figsize=(7, 5)) 
    plt.title(fileNames[i]+a)
    plt.plot(ss(i,k,bounds[i])[0],z[0]*ss(i,k,bounds[i])[0]+z[1],'royalblue',linewidth=2,label='Fit')
    plt.plot(ss(i,k,bounds[i])[0],ss(i,k,bounds[i])[1],'k.',linewidth=3,label='Experimental')
    plt.xlabel('Strain')
    plt.ylabel('Stress')
    plt.figtext(0.4,0.2,"Young's Modulus = " + str(round(z[0],6)) +" GPa, " + "r = " + str(round(z[2],5)))
    plt.legend()
    plt.savefig(graphPath + "_" + fileNames[i] + a + ".pdf", bbox_inches = "tight")
    plt.close()
    return i

In [92]:
'''function that calculates the average Young's Modulus with 95% Confidence Interval from all files'''

def stat(*args):
    l = args
    tt = t.ppf(0.95,numFiles-1)
    σ = tstd(l)
    μ = tmean(l)
    cl = tt*σ/numFiles**0.5
    #print(μ(average), "+/-", cl)
    return [μ, cl]

In [93]:
def excel(names, k):
    
    fileNames = names
    fileNames = np.append(fileNames,' ')
    fileNames = np.append(fileNames,'Average YM')
    fileNames = np.append(fileNames,'95% CI')
    excelpath = excelPath + folderNames[k] + '1.xlsx'
    
    writer = pd.ExcelWriter(excelpath)
    workbook = writer.book
    df  = pd.DataFrame(fileNames)
    df2 = pd.DataFrame(YM)
    df3 = pd.DataFrame(r)
    df4 = pd.DataFrame(statYM)

    headerYM = ['YM1','YM2']
    headerr = ['r1', 'r2']

    writer.sheets["Data"] = workbook.add_worksheet("Data")

    df2 = df2.transpose()
    df3 = df3.transpose()
    df4 = df4.transpose()

    df.to_excel(writer, sheet_name = "Data", index = False, header = ["File"], startcol = 0)  #saves the file names 
    df2.to_excel(writer, sheet_name = "Data", index = False, header = headerYM, startcol = 1) 
    df3.to_excel(writer, sheet_name = "Data", index = False, header = headerr, startcol = 3)
    df4.to_excel(writer, sheet_name = "Data", index = False, header = False, startrow = (numFiles+2), startcol = 1)

    writer.save()
    return k

In [94]:
'''applies functions to solve YM for different regions of the stress/strain curve
print graphs of the YM fitting the different regions of the stress/strain curve'''

for k in range(len(folderNames)):
    fileNames = sorted(os.listdir(folderpath + '/'+ folderNames[k]))
    e = -1
    for i in range(len(fileNames)):
        if (fileNames[i] == '.DS_Store'):
            e = i
    if (e >= 0):
        fileNames.remove('.DS_Store')
    print(fileNames)
    numFiles = len(fileNames)
    YM = np.zeros((numRanges,numFiles))
    r = np.zeros((numRanges,numFiles))
    peakpoint = np.zeros(numFiles)
    bounds = np.zeros((numFiles,numRanges))
    for j in range(numRanges):
        temp_a = graphRanges[j][0]
        temp_b = graphRanges[j][1]
        for i in range(numFiles):
            peakpoint[i] = peak(i,k)
            #print(i,j,k)
            bounds[i] = curve(i,k,temp_a,temp_b)
            graphs(k,i,j)
            YM[j][i] = ym(i,k)[0]
            r[j][i] = ym(i,k)[1]
        statYM[j] = stat(YM[j])
    excel(fileNames,k)


['Anode_N_C_40samples_1_1.csv', 'Anode_N_C_40samples_2_1.csv', 'Anode_N_C_40samples_3_1.csv', 'Anode_N_C_40samples_4_1.csv']
0 0 0
9098
6303
1 0 0
9233
6409
2 0 0
9181
6243
3 0 0
9236
6273
0 1 0
9098
7284
1 1 0
9233
7421
2 1 0
9181
7356
3 1 0
9236
7414
['Anode_N_U_48samples1_1_1.csv', 'Anode_N_U_48samples1_2_1.csv', 'Anode_N_U_48samples1_3_1.csv', 'Anode_N_U_48samples1_4_1.csv']
0 0 1
6242
2687
1 0 1
6711
3336
2 0 1
6302
2986
3 0 1
6185
3092
0 1 1
6242
4353
1 1 1
6711
4796
2 1 1
6302
4423
3 1 1
6185
4319
['Anode_O_C_48samples_1_1.csv', 'Anode_O_C_48samples_2_1.csv', 'Anode_O_C_48samples_3_1.csv', 'Anode_O_C_48samples_4_1.csv']
0 0 2
8387
5199
1 0 2
8733
5466
2 0 2
8610
5545
3 0 2
9232
6114
0 1 2
8387
6551
1 1 2
8733
6906
2 1 2
8610
6783
3 1 2
9232
7397
['Anode_O_U_48samples_1_1.csv', 'Anode_O_U_48samples_2_1.csv', 'Anode_O_U_48samples_3_1.csv', 'Anode_O_U_48samples_4_1.csv']
0 0 3
9015
4486
1 0 3
7894
3624
2 0 3
6754
3119
3 0 3
8291
4847
0 1 3
9015
7063
1 1 3
7894
5968
2 1 3
6754
4840


In [80]:
sorted(os.listdir('/Users/andreaithompson/Documents/ResearchBMR/MojdehTests/YoungModulus/JunTests/RawData'))

['.DS_Store',
 'Anode_N_C_40samples1',
 'Anode_N_U_48samples1',
 'Anode_O_C_48samples1',
 'Anode_O_U_48samples1',
 'HE5050_C_48samples1',
 'HE5050_U_48samples1',
 'LCO_C_48samples1',
 'LCO_U_48samples1',
 'NMC_C_47samples1',
 'NMC_U_48samples1']

In [82]:
sorted(os.listdir('/Users/andreaithompson/Documents/ResearchBMR/MojdehTests/YoungModulus/JunTests/RawData/Anode_N_U_48samples1'))

['Anode_N_U_48samples1_1_1.csv',
 'Anode_N_U_48samples1_2_1.csv',
 'Anode_N_U_48samples1_3_1.csv',
 'Anode_N_U_48samples1_4_1.csv']